In [1]:
#!pip install opencv-python
#!apt-get update && apt-get install ffmpeg libsm6 libxext6  -y
#!pip install mediapipe

In [2]:
#!pip install opencv-python transformers accelerate


In [3]:
import time
t = str(int(time.time()))+"_"
t

'1728661963_'

In [4]:
from diffusers import AutoencoderKL, StableDiffusionXLControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import torch
from controlnet_aux import OpenposeDetector
from diffusers.utils import load_image
from diffusers import ControlNetModel, StableDiffusionXLControlNetPipeline, AutoencoderKL
from diffusers import DDIMScheduler, EulerAncestralDiscreteScheduler
from controlnet_aux import MidasDetector

from PIL import Image
import torch
import random
import numpy as np
import cv2

2024-10-11 15:52:49.978395: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 15:52:49.978469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 15:52:49.978509: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import os
import cv2
import numpy as np
from PIL import Image
from controlnet_aux import MidasDetector
from pathlib import Path

# Укажите папки с изображениями и для сохранения масок
input_folder = './src/'
output_folder = './masks/'

# Убедитесь, что папка для масок существует
Path(output_folder).mkdir(parents=True, exist_ok=True)

# Инициализация модели
processor_midas = MidasDetector.from_pretrained("lllyasviel/Annotators", cache_dir='./modele cache/')

# Обход всех файлов в папке input_folder
for filename in os.listdir(input_folder):
    if filename.endswith((".JPG", ".png")):  # Можно добавить другие форматы изображений
        img_path = os.path.join(input_folder, filename)
        
        # Чтение изображения
        img = cv2.imread(img_path)

        # Размер изображения
        original_height, original_width = img.shape[:2]
        
        # Определяем соотношение сторон и изменяем размер так, чтобы одна сторона стала 512 пикселей
        if original_width > original_height:
            new_width = 512
            new_height = int(original_height * 512 / original_width)
        else:
            new_height = 512
            new_width = int(original_width * 512 / original_height)
        
        # Изменяем размер изображения с сохранением пропорций
        resized_img = cv2.resize(img, (new_width, new_height))
        

        # Добавляем отступы, используя cv2.BORDER_REPLICATE для копирования последней полоски пикселей
        padded_img = cv2.copyMakeBorder(
            resized_img, 
            (512 - new_height) // 2, (512 - new_height + 1) // 2,  # Верх и низ
            (512 - new_width) // 2, (512 - new_width + 1) // 2,    # Лево и право
            cv2.BORDER_REPLICATE  # Копируем крайние пиксели
            )

        
        
        # Обработка изображения с помощью Midas
        controlnet_img = processor_midas(padded_img, output_type='cv2')
        controlnet_img_pil = Image.fromarray(controlnet_img)
        
        # Сохранение маски в output_folder
        mask_path = os.path.join(output_folder, f"{filename.split('.')[0]}_m_controlnet.jpg")
        controlnet_img_pil.save(mask_path)
        
        print(f"Обработано и сохранено: {mask_path}")

print("Все изображения обработаны.")

/root/miniconda3/lib/python3.11/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


Обработано и сохранено: ./masks/IMG_0112_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0209_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0214_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0177_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0084_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0067_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0294_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0302_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0336_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0350_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0120_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0031_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0374_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0168_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0348_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0164_m_controlnet.jpg
Обработано и сохранено: ./masks/IMG_0163_m_controlnet.jpg
Обработано и с

In [6]:
filename

'IMG_0396.JPG'

In [1]:
!nvidia-smi

Fri Oct 11 16:28:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 Ti     Off | 00000000:83:00.0 Off |                  N/A |
| 57%   73C    P2             164W / 165W |  10259MiB / 16380MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--